### Importações necessárias

In [1]:
import numpy as ny
import math
from numpy import *
from numpy.linalg import inv
import numpy.linalg as la

### Função de impressão tester

In [2]:
def pmatriz(matrizz,mens):
    matrizz=ny.array(matrizz)
    print('*='*50)
    print(f'{mens:^90}')
    print('*='*50)
    print('     ',end='')
    for i in range(1,12):
        print(f'{i:^8}', end='')
    print('       ','*='*50)
    for i in range(matrizz.shape[0]):
        #print(f'{i+1:>2} == ', end='')
        for j in range(matrizz.shape[1]):
            print(f'{matrizz[i][j]:>5.2f} ', end='')
        print()
    print('*='*50)  

### Criando a matriz[n][m] de respostas a partir dos dados fornecidos pelo mestre Altobelli.

In [21]:
arq = open('bloodPressureNishisatobook-base.D18.N15.txt', 'r')
texto = arq.readlines()
matriz=[]
lres=[]
nrpp = 3
for linha in texto:
    limite = 0
    linElem = linha.split()
    while limite < len(linElem):
        a = int(linElem[limite]) - nrpp * limite
        if (a % 3 == 0):
            lres += [0,0,1]
        elif (a % 2 == 0):
            lres += [0,1,0]
        else:
            #from IPython.core.debugger import Tracer; Tracer()()
            lres += [1,0,0]
        limite += 1
    matriz += [lres]
    lres=[]
arq.close()
MPR = ny.matrix(matriz)
#print(*matrizF,sep='\n')
ncol = MPR.shape[1]
nlin = MPR.shape[0]
nperguntas = ncol//nrpp
Des=ncol-nperguntas-1
#print('\nNúmero de colunas = ',ncol)
#print('Número de linhas = ',nlin)
#print('Número de perguntas = ',nperguntas)
#print('Dimensões do espaço solução = ',Des)

### Matriz transposta da MPR

In [22]:
MPRt = ny.transpose(MPR) 
#print(MPRt.shape)

### Vetor de frequência de linhas (Vfl)

In [23]:
#Cálculo da Matriz Diagonal de Linhas de F
Vfl = ny.sum(MPR, axis=1)
#print (Vfl)

### Vetor de frequência de colunas (Vfcc)

In [24]:
Vfcc=[]
soma=0
for i in range(ncol):
    for j in range(len(matriz)):
        soma+=matriz[j][i]
    Vfcc.append(soma)
    soma = 0
Vfc = ny.sum(MPR, axis=0)
#print(Vfc.shape,'\n')
#print(len(Vfcc))
#print(Vfc,'\n\n',Vfcc)

### Cálculo das matrizes diagonais - MdL = matriz diagonal de linhas(nxn) - MdC = matriz diagonal de colunas(mxm)

In [25]:
MdC = ny.diagflat(Vfc)
MdL = ny.diagflat(Vfl)
#print('   Matriz Diagonal de Linhas')
#print(*MdL, sep='\n')
#print('     Matriz Diagonal de Colunas')
#print(*MdC, sep = '\n')

### Matriz de correlação entre colunas

In [26]:
MdCi = inv(MdC)
MdLi = inv(MdL)
Mcc = MPRt * MdLi * MPR * MdCi 
#pmatriz(Mcc,'Matriz de Correlação entre Colunas')

### Vetor de autovalores(Vetaval) e matriz de autovetores(Mavet) com normalização

In [27]:
vetaval,mavet=la.eig(Mcc)
Vetaval=ny.flip(ny.sort(vetaval, axis=0),0)
valaux=mavet.T.tolist() # transforma uma matriz numpy em uma lista para facilitar a manipulação dos dados.
Mavet=[] # Cria a Matriz de autovetores ordenada em função dos autovalores
for elem in Vetaval:
    ix = vetaval.tolist().index(elem)
    Mavet.append(valaux[ix])
Mavet=ny.array(Mavet).T.astype(ny.float32) # converte números complexos para float 32
Vetaval=Vetaval.astype(ny.float32)
#'\n',Vetaval)
Vetaval=ny.delete(Vetaval,0,0) # apaga a solução trivial no vetor de autovalores e na matriz
Mavet = ny.delete(Mavet,0,1)   # de autovetores removendo a primeira dimensão
while Des < Vetaval.shape[0]:
    Vetaval = ny.delete(Vetaval,Des,0)
    Mavet = ny.delete(Mavet,Des,1)

C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  
C:\Users\JRicardo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


### Cálculo do Rho

In [28]:
rho=list()
for x in Vetaval:
    rho.append(sqrt(x))
rho=ny.array(rho)
#for i in range(len(rho)):
 #   print(f'{rho[i]:>5.4f}  ', end='')

### Matriz resultante do porduto de Hadamard(MpHad)

In [29]:
MpHad=ny.multiply(Mavet,Mavet)
#print(MpHad)

### Calculando a Matriz T e seu vetor de frequência de colunas

In [30]:
matrizT = MdC * MpHad #equação(7)
matrizT = ny.array(matrizT)
VfcT=list()
VfcT=ny.sum(matrizT, axis=0) #soma coluna, caso axis=1 somaria linha = equação(8)
#for i in VfcT:
    #print(f'{i:>5.4f} ', end='')

### Vetor de multiplicadores e escalar da matriz padrão-de-respostas. eq(9)

In [31]:
escalarMPR=ny.sum(matriz)
VetMulticT=list()
for i in VfcT:
    VetMulticT.append(math.sqrt(escalarMPR/i))
#for i in range(len(VetMulticT)):
 #   print(f'{VetMulticT[i]:>7.4f}', end='')

### Cálculo da Matriz de Pesos Padrão dos itens(Mppi = x_normed)

In [32]:
# Equação(10) Matriz de pesos padrão dos itens(Mppi)
Mppi=list()
aux=list()
for i in range(Mavet.shape[0]):
    for n in range(len(VetMulticT)):
        aux.append(Mavet[i][n]*VetMulticT[n])
    Mppi.append(aux[:])
    aux.clear()
Mppi = ny.array(Mppi)
#pmatriz(Mppi,'Matriz de Pesos Padrão dos Itens')

### Coordenadas dos itens nas Dimensões (Mcides = x_projected)

In [33]:
Mcides = Mppi * rho
#pmatriz(Mcides,'Matriz de Coordenadas dos Itens nas Dimensões')

### Coordenadas das transações nas Dimensões

In [34]:
Ctdes = MPR * Mppi # fórmula (13)
Ctdes = ny.array(Ctdes)
#pmatriz(Ctdes,'Matriz de Coordenadas das Transações nas Dimensões')

### Matriz de Pesos Padrão das Transações (Mppt = y_normed)

In [35]:
aux= Vfl * rho
Mppt = Ctdes/aux # fórmula (14)
#pmatriz(Mppt,'Matriz de Pesos Padrão de Transações')

### Matriz de Coordenadas das Transações nas Dimensões (Mctd = y_projected)

In [36]:
Mctd = Ctdes / Vfl # fórmula (15) simplificada
Mctd = ny.array(Mctd)
#pmatriz(Mctd,'Matriz de Coordenadas das Transações de itens')

### Calculando o Delta

In [37]:
Delta = Vetaval * (100 / ny.sum(Vetaval))
#for i in Delta:
 #   print(f'{i:5.4f}  ', end='')

### Selecionando as relações com Confiança igual ou maior que 40%

In [38]:
arq2 = open('bloodPressureNishisatobook.D18.N15.txt', 'r')
texto2 = arq2.readlines()
arq2.close()
confList = []
confDesejada = 0.8 # 40% de confiança
for x in texto2:
    linhatxt = x.split()
    conflinha = float(linhatxt[len(linhatxt)-1])
    if conflinha >= confDesejada:
        confList.append(linhatxt[:])
antcd = []
consq = []
aux1 = []
aux2 = []
for y in confList:
    ind = y.index('==>')
    for ini in range(len(y)):
        if y[ini]=='#SUP:':
            consq.append(aux2[:])
            aux2.clear()
            ini=len(y)
            break
        elif ini < ind:
            aux1.append(int(y[ini]))
            next
        elif y[ini]=='==>':
            if len(aux1) > 1:
                antcd.append(aux1[:])
            else:
                antcd.append(aux1[:])
            aux1.clear()
            next
        elif ini > ind:
            aux2.append(int(y[ini]))
#for i in range(len(antcd)):
    #confianca = float(confList[i][len(confList[i])-1])*100
    #print(f'projeção de {antcd[i]} em {consq[i]} apresenta confiança de {confianca:5.1f}%.')
#for y in confList:
 #   a=float(y[len(y)-1])*100
  #  print(f'Confiança de {a:5.1f}%')
print()

### Função para cálculo da distância entre itens

In [39]:
def dist2ptsi(ptos):
    ematriz = (len(ptos) == Mcides.shape[0])
    if ematriz: Ditens = ny.zeros((len(ptos),len(ptos)), dtype=float32) # matriz de zeros
    if len(ptos) < 2:
        return 'Hove um erro e foi passado apenas um ponto!'
    i=0
    while len(ptos) >= 2:
        while i in range(len(ptos)):
            for j in range(1,len(ptos)):
                d_itens = 0
                linha = ptos[0]-1
                coluna = ptos[j]-1
                for k in range(Des):
                    d_itens += rho[k] * (((Mcides[linha][k])/math.sqrt(Vfcc[linha]/nlin) 
                              - (Mcides[coluna][k])/math.sqrt(Vfcc[coluna]/nlin))**2)
                if ematriz:
                    Ditens[linha,coluna] = round(d_itens,2)
                    Ditens[coluna,linha] = round(d_itens,2)
                else: print(f'Distância entre {ptos[i]} e {ptos[j]} = {d_itens:.2f}')
            ptos.pop(0)
    if ematriz: return Ditens

### Cáclulo da distância entre itens

In [40]:
listptsi=list() # lista de regras de confiança para projeção com mais de um ponto.
'''for i in range(len(consq)):
    if (len(consq[i]))>=2:
        for aux in [i]:
            listptsi += [aux]          
for i in listptsi:
    print(f'Para o vetor {consq[i]}, temos:')
    dist2ptsi(consq[i][:])
    print()'''
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
result = dist2ptsi(a[:])
resultado = list()
for i in range(len(result)):
    resultado.append(ny.sum(result[i]/18))
#pmatriz(result,'Distância entre todos os itens')
print(*resultado)
#c = [4,7,8]
#dist2ptsi(c[:])

7.532778 6.427778 8.03389 7.9444447 7.9894447 6.035 7.8372226 6.788889 7.5133333 8.292778 7.7388887 5.4911113 5.8833337 7.867778 7.616111 6.7583337 6.4738894 6.741667


### Função para cálculo da distância entre transações

In [41]:
def dist2ptst(ptos):
    ematriz = (len(ptos) == Mctd.shape[0])
    if ematriz: 
        print('é matriz')
        Ditens = ny.zeros((len(ptos),len(ptos)), dtype=float32) # matriz de zeros
    if len(ptos) < 2:
        return 'Hove um erro e foi passado apenas um ponto!'
    i=0
    while len(ptos) >= 2:
        while i in range(len(ptos)):
            for j in range(1,len(ptos)):
                d_itens = 0
                linha = ptos[0]-1
                coluna = ptos[j]-1
                for k in range(Des):
                    d_itens += rho[k] * (((Mctd[linha][k]/math.sqrt(Vfl[linha]/ncol))
                                          - (Mctd[coluna][k]/math.sqrt(Vfl[coluna]/ncol)))**2)
                if ematriz:
                    Ditens[linha,coluna] = round(d_itens,2)
                    Ditens[coluna,linha] = round(d_itens,2)
                else: print(f'Distância entre {ptos[i]} e {ptos[j]} = {d_itens:.2f}')
            ptos.pop(0)
    if ematriz: return Ditens

### Cáclulo da distância entre  as transações

In [42]:
'''listptst=list() # lista de regras de confiança para projeção com mais de um ponto.
for i in range(len(pem)):
    if (len(pem[i]))>=2:
        for aux in [i]:
            listptst += [aux]'''          
#for i in listptst:
    #print(f'Para o vetor {pem[i]}, temos:')
    #dist2ptst(pem[i][:])
    #print()
#b = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
#b = ny.array(b)
#pmatriz(dist2ptst(b[:]),'Distância entre todos os itens')
#c = [4,7,8]
#dist2ptst(c[:])
print()